In [ ]:
# Lab 5: Gradio UI + 최종 통합 (40분)
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- Gradio를 활용한 웹 UI 구축
- Lab 1-4의 모든 기능을 UI에 통합
- 실전 민원 처리 시스템 완성
"""

# ============================================================
# 환경 설정
# ============================================================
import sys,os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber gradio -q

import getpass
from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
import gradio as gr
from typing import List, Dict
from datetime import datetime

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = "".strip()
client = OpenAI(api_key=api_key)


print("[완료] 환경 설정 완료!\n")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.6 MB/s eta 0:00

In [2]:
# ============================================================
# Lab 1-4 함수 통합 (빠른 재구축)
# ============================================================

print("\n[진행중] 시스템 초기화 중...")

# 기본 함수들
def classify_complaint(text: str) -> Dict[str, str]:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )
    return {"원문": text, "분류_결과": response.choices[0].message.content}

def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    pages_data = []
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))
        for i in range(total_pages):
            text = pdf.pages[i].extract_text()
            pages_data.append({"page_num": i + 1, "text": text, "char_count": len(text)})
    return pages_data

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    chunks = []
    chunk_id = 0
    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            if chunk_text.strip():
                chunks.append({
                    "id": f"chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1
            start = end - overlap
    return chunks

def get_embedding(text: str) -> List[float]:
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# Vector DB 구축
print("[진행중] Vector DB 구축 중... (1-2분 소요)")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

try:
    chroma_client.delete_collection("fire_law")
    chroma_client.delete_collection("fire_prevention")
except:
    pass

# 소방기본법
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=10)
fire_law_chunks = chunk_text(fire_law_pages)
fire_law_collection = chroma_client.create_collection("fire_law")
for chunk in fire_law_chunks:
    embedding = get_embedding(chunk['text'])
    fire_law_collection.add(
        ids=[f"law_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "소방기본법"}]
    )

# 화재예방
prevention_pages = extract_text_from_pdf("화재예방.pdf")
prevention_chunks = chunk_text(prevention_pages)
prevention_collection = chroma_client.create_collection("fire_prevention")
for chunk in prevention_chunks:
    embedding = get_embedding(chunk['text'])
    prevention_collection.add(
        ids=[f"prev_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{"page": chunk['page'], "source": "화재예방가이드"}]
    )

# 검색 함수
def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    query_embedding = get_embedding(query)
    results = fire_law_collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return [
        {"text": results['documents'][0][i], "page": results['metadatas'][0][i]['page'], "source": results['metadatas'][0][i]['source']}
        for i in range(len(results['documents'][0]))
    ]

def search_fire_prevention(query: str, n_results: int = 3) -> List[Dict]:
    query_embedding = get_embedding(query)
    results = prevention_collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return [
        {"text": results['documents'][0][i], "page": results['metadatas'][0][i]['page'], "source": results['metadatas'][0][i]['source']}
        for i in range(len(results['documents'][0]))
    ]

def search_both_sources(query: str, n_results_per_source: int = 2) -> Dict:
    return {
        "법령": search_fire_law(query, n_results_per_source),
        "예방가이드": search_fire_prevention(query, n_results_per_source),
        "total_count": n_results_per_source * 2
    }

# 보고서 생성
def generate_complaint_report(complaint_text: str) -> Dict:
    classification = classify_complaint(complaint_text)
    search_results = search_both_sources(complaint_text, n_results_per_source=2)

    law_context = "\n".join([f"- {r['source']} {r['page']}p: {r['text'][:200]}" for r in search_results['법령']])
    prevention_context = "\n".join([f"- {r['source']} {r['page']}p: {r['text'][:200]}" for r in search_results['예방가이드']])

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방청 민원 처리 담당자입니다.
다음 형식으로 공식 보고서를 작성하세요:

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
【민원 처리 보고서】
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. 민원 내용
   [민원 요약]

2. 분류 결과
   [카테고리, 긴급도, 키워드]

3. 관련 법령 및 근거
   [소방기본법 관련 내용]

4. 처리 방안
   [구체적인 대응 방법]

5. 예방 가이드
   [화재예방 참고사항]

6. 담당 부서 및 처리 기한
   [추천 담당 부서 및 기한]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
작성일시: [현재 날짜]
작성자: 소방청 담당자
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"""
            },
            {
                "role": "user",
                "content": f"""다음 민원에 대한 처리 보고서를 작성하세요.

[민원 내용]
{complaint_text}

[분류 정보]
{classification['분류_결과']}

[참고 - 소방기본법]
{law_context}

[참고 - 화재예방 가이드]
{prevention_context}"""
            }
        ],
        temperature=0.4,
        max_tokens=1000
    )

    report = response.choices[0].message.content

    return {
        "민원원문": complaint_text,
        "분류": classification,
        "법령검색": search_results['법령'],
        "예방검색": search_results['예방가이드'],
        "보고서": report
    }

print("[완료] 시스템 준비 완료!\n")




[진행중] 시스템 초기화 중...
[진행중] Vector DB 구축 중... (1-2분 소요)
[완료] 시스템 준비 완료!



In [3]:
# ============================================================
# 실습 1: Gradio UI 함수
# ============================================================

print("="*60)
print("[실습 1] Gradio UI 함수 구현")
print("="*60)

def process_complaint_ui(complaint_text: str) -> str:
    """
    Gradio UI용 래퍼 함수

    입력: 민원 내용 (텍스트)
    출력: 처리 보고서 (텍스트)
    """
    # 입력 검증
    if not complaint_text or not complaint_text.strip():
        return "[경고] 민원 내용을 입력해주세요."

    if len(complaint_text) < 10:
        return "[경고] 민원 내용이 너무 짧습니다. 최소 10자 이상 입력해주세요."

    try:
        # 보고서 생성
        result = generate_complaint_report(complaint_text)
        return result['보고서']

    except Exception as e:
        return f"""[오류 발생]

오류 내용: {str(e)}

해결 방법:
1. API 키가 유효한지 확인하세요
2. PDF 파일이 올바르게 업로드되었는지 확인하세요
3. 인터넷 연결을 확인하세요
4. 민원 내용을 다시 입력해보세요"""

print("\n[완료] UI 함수 구현 완료")



[실습 1] Gradio UI 함수 구현

[완료] UI 함수 구현 완료


In [4]:
# ============================================================
# 실습 2: Gradio 인터페이스 구축
# ============================================================

print("\n" + "="*60)
print("[실습 2] Gradio 인터페이스 구축")
print("="*60)

# 예시 민원들
examples = [
    ["아파트 복도에 소화기가 없는데 괜찮은가요?"],
    ["식당을 운영하는데 소방시설 점검은 언제 받아야 하나요?"],
    ["소방차 전용구역에 주차했다가 과태료를 받았습니다."],
    ["회사 사무실 소방시설 정기 점검 주기가 궁금합니다."],
    ["주택에 화재경보기 설치가 의무인가요?"]
]

# Gradio 인터페이스 생성
interface = gr.Interface(
    fn=process_complaint_ui,
    inputs=gr.Textbox(
        label="민원 내용",
        placeholder="처리할 민원 내용을 입력하세요...\n\n예시:\n- 아파트 복도에 소화기가 없습니다\n- 식당 소방시설 점검 주기가 궁금합니다\n- 소방차 전용구역 주차 과태료 문의",
        lines=7,
        max_lines=10
    ),
    outputs=gr.Textbox(
        label="처리 보고서",
        lines=25,
        max_lines=30
    ),
    title="소방청 민원처리 AI Agent",
    description="""
    ### 민원 내용을 입력하면 자동으로 처리 보고서를 생성합니다

    **자동 수행 기능:**
    1. 민원 자동 분류 (카테고리, 긴급도, 키워드)
    2. 관련 법령 검색 (소방기본법)
    3. 예방 가이드 검색 (화재예방)
    4. 공식 처리 보고서 생성

    **사용 방법:**
    - 아래 예시를 클릭하거나 직접 민원을 입력하세요
    - 생성 버튼을 누르면 약 10-20초 후 보고서가 생성됩니다
    """,
    examples=examples,
    theme=gr.themes.Soft(),
    article="""
    ---
    ### 시스템 정보
    - **모델:** GPT-4o-mini
    - **임베딩:** text-embedding-3-small
    - **Vector DB:** ChromaDB
    - **데이터:** 소방기본법 (10p) + 화재예방 가이드 (44p)

    ### 주의사항
    - 생성된 보고서는 참고용이며, 최종 결정은 담당자 검토 후 확정됩니다
    - 법령은 최신 개정본을 별도로 확인하세요
    - 긴급 상황은 119로 즉시 신고하세요
    """
)

print("\n[완료] Gradio 인터페이스 구축 완료")




[실습 2] Gradio 인터페이스 구축

[완료] Gradio 인터페이스 구축 완료


In [5]:
# ============================================================
# 실습 3: UI 실행 및 테스트
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] Gradio UI 실행")
print("="*60)

print("""
[안내] Gradio UI를 실행합니다.

실행 후:
1. 자동으로 public URL이 생성됩니다
2. URL을 클릭하여 웹 인터페이스를 엽니다
3. 민원을 입력하고 테스트합니다

Colab 환경에서는:
- share=True로 설정하여 외부 접속 가능한 링크 생성
- 링크는 72시간 동안 유효
- 다른 사람들과 공유 가능
""")





[실습 3] Gradio UI 실행

[안내] Gradio UI를 실행합니다.

실행 후:
1. 자동으로 public URL이 생성됩니다
2. URL을 클릭하여 웹 인터페이스를 엽니다
3. 민원을 입력하고 테스트합니다

Colab 환경에서는:
- share=True로 설정하여 외부 접속 가능한 링크 생성
- 링크는 72시간 동안 유효
- 다른 사람들과 공유 가능



In [6]:
# ============================================================
# 최종 실행
# ============================================================

print("\n" + "="*60)
print("[완료] Lab 5 완료!")
print("="*60)
print("""
학습 내용:
1. Gradio 기본 사용법
2. UI 함수 구현 (입력 검증, 오류 처리)
3. 인터페이스 구성 (입력/출력/예시)
4. 웹 UI 배포 (share=True)

최종 시스템:
- 민원 입력 → 자동 분류 → 법령 검색 → 보고서 생성
- 웹 인터페이스로 누구나 쉽게 사용 가능
- 외부 공유 가능한 링크 생성

실전 활용:
1. 부서 내 공유하여 실무에 활용
2. 다양한 민원으로 테스트하여 정확도 검증
3. 필요시 프롬프트 수정하여 맞춤화

전체 Lab 완료!
Lab 1: API 기초
Lab 2: RAG 기초
Lab 3: 고급 검색
Lab 4: 보고서 생성
Lab 5: Gradio UI ← 현재

이제 실전 민원을 처리해보세요!
""")

# Gradio 실행
print("\n[실행] Gradio UI 시작...")
interface.launch(share=True, debug=True)


[완료] Lab 5 완료!

학습 내용:
1. Gradio 기본 사용법
2. UI 함수 구현 (입력 검증, 오류 처리)
3. 인터페이스 구성 (입력/출력/예시)
4. 웹 UI 배포 (share=True)

최종 시스템:
- 민원 입력 → 자동 분류 → 법령 검색 → 보고서 생성
- 웹 인터페이스로 누구나 쉽게 사용 가능
- 외부 공유 가능한 링크 생성

실전 활용:
1. 부서 내 공유하여 실무에 활용
2. 다양한 민원으로 테스트하여 정확도 검증
3. 필요시 프롬프트 수정하여 맞춤화

전체 Lab 완료!
Lab 1: API 기초
Lab 2: RAG 기초
Lab 3: 고급 검색
Lab 4: 보고서 생성
Lab 5: Gradio UI ← 현재

이제 실전 민원을 처리해보세요!


[실행] Gradio UI 시작...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d7dde2f65acabf3c27.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d7dde2f65acabf3c27.gradio.live
